In [1]:
import numpy as np
from midiutil import MIDIFile
import pygame
from qiskit import *
import random

pygame 2.0.0.dev1 (SDL 2.0.9, python 3.7.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


First I'll make a dictionary so that I can look up the MIDI addresses of notes.

In [2]:
note_ref = {'C':24,'C#':25,'D':26,'D#':27,'E':28,'F':29,'G#':30,'G':31,'G#':32,'A':33,'A#':34,'B':35}

Next I'll make a function that creates a list of bit strings, such that each string is only one bit different from it's neighbours in the list. For example, `make_line(8)` generates

    ['000', '100', '110', '010', '011', '111', '101', '001']

In [3]:
def make_line ( length ):
    # outputs `line`: a list of `length` bit strings (plus a few extra if length isn't a power of two)
    # strings defined such that line[j] is Hamming distance 1 from line[j+1], for all j
    n = int(np.ceil(np.log(length)/np.log(2)))
    line = ['0','1']
    for j in range(n-1):
        line = line + line[::-1]
        for j in range(int(len(line)/2)):
            line[j] += '0'
        for j in range(int(len(line)/2),int(len(line))):
            line[j] += '1'
    return line

The idea behind what follows is to take a score and represent each event using the information

* note
* octave
* bar
* beat
* volume

I then create a separate subscore for each note. For each event in this, I treat (octave,bar,beat) as a coordinate.

To each set of coordinates I associate a bit string, such that these strings differ by only one bit if they correspond to an event on the same beat in the same bar, but on a neighbouring octave, or the same octave and bar but a neighbouring beat, or the same octave and beat but a neighbouring bar. This can be done by using `make_line` to create a seperate line for octaves, bars and beats, and then associating a bit string with each octave, beat and bar. The bit string for any (octave,bar,beat) is then chosen to be the concatenation of the octave, beat and bar strings.

I then encode the subscore as a quantum state, by associating the probability of the output bit string for (octave,beat,bar) with the corresponding volume. Encoding the subscore can be done by creating a statevector for which the amplitudes correspond to the square root of the volume (and then normalized). Decoding can be done straightforwardly from the counts dictionary (given enough samples). On a simulator, it can be done by extracting the statevector and reversing the encoding.

While the subscore is in the quantum program, it can be manipulated. Small angle rotations, for example, will cause events to bleed volume into neighbouring coordinates and begin to quantumly interefere with each other.

The above is all the idea behind what follows. Unfortunately, I seem to have strayed from this when actually doing it. It does produce some sort of an output, though!

The parameter `theta` below affects how much interference is caused. For `theta = 0`, the score comes out as it went in. For `theta = np.pi/2` it is a total keyboard mash.

In [4]:
theta = np.pi/3

In [5]:
def make_box ( lengths ):
    # same as `make_line`, but outputs `box` for multidimensonal connectivity
    box = ['']
    for length in lengths:
        new_box = []
        line = make_line(length)
        for bstring in box:
            for lstring in line:
                new_box.append( bstring + lstring)
        box = new_box
    return box

In [6]:
# set up a piece
bar_num = 12     # number of bars
octave_num = 4    # number of octaves
beat_num = 4      # number of beats per bar, (or half beats, etc depending on desired resolution and tempo)
notes_used = ['C','D','E','F','G','A']   # ['C','C#','D','D#','E','F','G#','G','G#','A','A#','B']

In [7]:
# set up a score
score = []

score.append({'note':'C','octave':2,'bar':0,'beat':0, 'volume':100})
score.append({'note':'C','octave':2,'bar':0,'beat':1, 'volume':100})
score.append({'note':'G','octave':2,'bar':0,'beat':2, 'volume':100})
score.append({'note':'G','octave':2,'bar':0,'beat':3, 'volume':100})
score.append({'note':'A','octave':2,'bar':1,'beat':0, 'volume':100})
score.append({'note':'A','octave':2,'bar':1,'beat':1, 'volume':100})
score.append({'note':'G','octave':2,'bar':1,'beat':2, 'volume':100})
score.append({'note':'F','octave':2,'bar':2,'beat':0, 'volume':100})
score.append({'note':'F','octave':2,'bar':2,'beat':1, 'volume':100})
score.append({'note':'E','octave':2,'bar':2,'beat':2, 'volume':100})
score.append({'note':'E','octave':2,'bar':2,'beat':3, 'volume':100})
score.append({'note':'D','octave':2,'bar':3,'beat':0, 'volume':100})
score.append({'note':'D','octave':2,'bar':3,'beat':1, 'volume':100})
score.append({'note':'C','octave':2,'bar':3,'beat':2, 'volume':100})
score.append({'note':'G','octave':2,'bar':4,'beat':0, 'volume':100})
score.append({'note':'G','octave':2,'bar':4,'beat':1, 'volume':100})
score.append({'note':'F','octave':2,'bar':4,'beat':2, 'volume':100})
score.append({'note':'F','octave':2,'bar':4,'beat':3, 'volume':100})
score.append({'note':'E','octave':2,'bar':5,'beat':0, 'volume':100})
score.append({'note':'E','octave':2,'bar':5,'beat':1, 'volume':100})
score.append({'note':'D','octave':2,'bar':5,'beat':2, 'volume':100})
score.append({'note':'G','octave':2,'bar':6,'beat':0, 'volume':100})
score.append({'note':'G','octave':2,'bar':6,'beat':1, 'volume':100})
score.append({'note':'F','octave':2,'bar':6,'beat':2, 'volume':100})
score.append({'note':'F','octave':2,'bar':6,'beat':3, 'volume':100})
score.append({'note':'E','octave':2,'bar':7,'beat':0, 'volume':100})
score.append({'note':'E','octave':2,'bar':7,'beat':1, 'volume':100})
score.append({'note':'D','octave':2,'bar':7,'beat':2, 'volume':100})
score.append({'note':'C','octave':2,'bar':8,'beat':0, 'volume':100})
score.append({'note':'C','octave':2,'bar':8,'beat':1, 'volume':100})
score.append({'note':'G','octave':2,'bar':8,'beat':2, 'volume':100})
score.append({'note':'G','octave':2,'bar':8,'beat':3, 'volume':100})
score.append({'note':'A','octave':2,'bar':9,'beat':0, 'volume':100})
score.append({'note':'A','octave':2,'bar':9,'beat':1, 'volume':100})
score.append({'note':'G','octave':2,'bar':9,'beat':2, 'volume':100})
score.append({'note':'F','octave':2,'bar':10,'beat':0, 'volume':100})
score.append({'note':'F','octave':2,'bar':10,'beat':1, 'volume':100})
score.append({'note':'E','octave':2,'bar':10,'beat':2, 'volume':100})
score.append({'note':'E','octave':2,'bar':10,'beat':3, 'volume':100})
score.append({'note':'D','octave':2,'bar':11,'beat':0, 'volume':100})
score.append({'note':'D','octave':2,'bar':11,'beat':1, 'volume':100})
score.append({'note':'C','octave':2,'bar':11,'beat':2, 'volume':100})

In [8]:
bars = int(np.ceil(np.log(bar_num)/np.log(2)))
octaves = int(np.ceil(np.log(octave_num)/np.log(2)))
beats = int(np.ceil(np.log(beat_num)/np.log(2)))

box = make_box([bar_num,octave_num,beat_num])

ket = {}
for note in notes_used:
    ket[note] = [0]*len(box)

for event in score:
    address = ''
    for coord,limit in [(event['bar'],bars),(event['octave'],octaves),(event['beat'],beats)]:
        address += ("{:0"+str(limit)+"b}").format(coord)
    ket[event['note']][int(address,2)] = np.sqrt(event['volume'])
for note in notes_used:
    N = 0
    for amp in ket[note]:
        N += amp*np.conj(amp)
        
    for j,amp in enumerate(ket[note]):
        ket[note][j] = amp/np.sqrt(N)

In [9]:
n = bars+octaves+beats
backend = Aer.get_backend('qasm_simulator')

qcs = {}
for note in notes_used:
    qcs[note] = QuantumCircuit(n,n,name=note)
    qcs[note].initialize(ket[note],qcs[note].qregs[0])
    qcs[note].ry(theta,qcs[note].qregs[0])
    qcs[note].measure(qcs[note].qregs[0],qcs[note].cregs[0])

counts = {}
for note in notes_used:
    
    counts[note] = execute(qcs[note],backend,shots=8192).result().get_counts(note)

In [10]:
notes = {}

for note in notes_used:
    maxvol = 0
    these_notes = {}
    for string in counts[note]:
        beat = int(string[-beats::],2)
        octave = int(string[-beats-octaves:-beats:],2)
        bar = int(string[0:bars],2)
        vol = np.log( counts[note][string] )
        these_notes[(note,beat,octave,bar)] = vol
        maxvol = max(maxvol,vol)

    for coords in these_notes:
        these_notes[coords] = int(these_notes[coords]*100/maxvol)

    for note in these_notes:
        notes[note] = these_notes[note]      

In [11]:
track = 0
channel = 0
tempo = 60

MyMIDI = MIDIFile(1)
MyMIDI.addTempo(track, 0, tempo)
MyMIDI.addProgramChange(0,0,0,1)

for (note,beat,octave,bar) in notes:

    degree = note_ref[note] + octave*12
    time = beat + beat_num*bar
    duration = 1
    volume = notes[(note,beat,octave,bar)]
    MyMIDI.addNote(track, channel, degree, time, duration, volume)
    
with open("music.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

In [12]:
pygame.init()

pygame.mixer.music.load("music.mid")
pygame.mixer.music.play()